In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [3]:
import seaborn as sns                   #veri görselleştirilmesi için kullanılır
import matplotlib.pyplot as plt         #veri görselleştirilmesi için kullanılır
data_train=pd.read_csv("train.csv")     #csv dosyalarını okumak için kullanılır
data_test=pd.read_csv("test.csv")       #csv dosyalarını okumak için kullanılır

In [4]:
data_test['Machine failure']=np.nan     #test verisine "Machine failure" adında boş bir stun eklemek için kullanılmış
data_train['data']='train'              #data adında yeni bir stun eklen
data_test['data']='test'                #data adında yeni bir stun eklen
data_test=data_test[data_train.columns] #eğitim ve test verilerinin aynı sütun numarasına sahip olması sağlanmış

In [5]:
original_columns = [                                                    #orjinal stun adlarını belirlemek için kullanılmış
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]

new_columns = {col: col.split(' [')[0] for col in original_columns}     #yeni stun oluşturup stün adlarını daha anlamlı hale getirmek için kullanılır
data_train = data_train.rename(columns=new_columns)                     #data_train stün adlarıonı yeni stun isimleriyle değiştirir

data_test = data_test.rename(columns=new_columns)                       #aynı işlemi data_test içinde yapar
#print(data_test.columns)                                               #yapılan işlemlerin kontrolü sağlanır

In [6]:
data_all=pd.concat([data_train,data_test],axis=0)                       #Bütün verileri birleştirmek için kullanılır

In [7]:
data_train.isna().sum()                                                 #verilerimizde boş biryer olup olmadığı koıntrol edilir

id                     0
Product ID             0
Type                   0
Air temperature        0
Process temperature    0
Rotational speed       0
Torque                 0
Tool wear              0
Machine failure        0
TWF                    0
HDF                    0
PWF                    0
OSF                    0
RNF                    0
data                   0
dtype: int64

In [8]:
data_all['Machine failure'].value_counts()              #stünun içindeki hatalarıun sayısını belirlenir

Machine failure
0.0    134281
1.0      2148
Name: count, dtype: int64

In [9]:
134281/2148                                             #cihazın hata verme oranı hesaplanır

62.51443202979516

In [10]:
list(zip(data_all.columns,data_all.dtypes,data_all.nunique()))          #bütün değerler hesaplanır sırasıyla stün ismi, typı, kaç tane farklı değer olduğu hesaplanmıştır

[('id', dtype('int64'), 227383),
 ('Product ID', dtype('O'), 9995),
 ('Type', dtype('O'), 3),
 ('Air temperature', dtype('float64'), 95),
 ('Process temperature', dtype('float64'), 84),
 ('Rotational speed', dtype('int64'), 974),
 ('Torque', dtype('float64'), 628),
 ('Tool wear', dtype('int64'), 246),
 ('Machine failure', dtype('float64'), 2),
 ('TWF', dtype('int64'), 2),
 ('HDF', dtype('int64'), 2),
 ('PWF', dtype('int64'), 2),
 ('OSF', dtype('int64'), 2),
 ('RNF', dtype('int64'), 2),
 ('data', dtype('O'), 2)]

In [11]:
types=pd.get_dummies(data_all['Type'], drop_first=False)                #type stunundaki değerleri binary olarak dönüştürür (0 ve 1) (True or False)

#bunun yerine label encoder kullanılabilirdi
#from sklearn.preprocessing import LabelEncoder
#le=LabelEncoder()

#LeColum="Type"
#LeColum2="Product ID"
#data_all[LeColum] = le.fit_transform(data_all[LeColum])
#data_all[LeColum2] = le.fit_transform(data_all[LeColum2])

In [12]:
mapping_boolean = {True: 1, False: 0}                           #boolean değerlerini 0 ve 1 e dönüştürür
types['H'] = types['H'].map(mapping_boolean)
types['L'] = types['L'].map(mapping_boolean)
types['M'] = types['M'].map(mapping_boolean)

In [13]:
data_all=pd.concat([data_all, types], axis=1)                   #dönüştürülen boolean verisini bütün tabloya ekler

In [14]:
data_all.drop(['id', 'Product ID', 'Type'], axis=1, inplace=True)   #bütün veriden id product id ve typı siler

In [15]:
data_train=data_all[data_all['data']=='train']                      #data stünunun içndeki train olanları seçer
del data_train['data']                                              #data stununu kaldırır
data_test=data_all[data_all['data']=='test']                        #data stünunun içndeki test olanları seçer
data_test.drop(['Machine failure','data'],axis=1,inplace=True)      #'Machine failure','data' stunlarını kaldırır

C:\Users\Basri\AppData\Local\Temp\ipykernel_13268\455916649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop(['Machine failure','data'],axis=1,inplace=True)      #'Machine failure','data' stunlarını kaldırır


In [16]:
from sklearn.model_selection import train_test_split                                    #veriyi eğitim ve test kümelerine bölen bir fonksiyon çağırır

X = data_train.drop(columns="Machine failure")                                          #tahmin etmemizi istedikleri veri Machine daillure olduğu için bu veri çıkartılır
y = data_train["Machine failure"]                                                       #tahmin edeceğimiz veri ayrı seçilir


X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.3,random_state=0)       #train test kümelerine ayırır verinin %25 i test için ayırır random_state ise seed vermek için kullanılır

In [17]:
X_train.shape, X_val.shape              #ayrılan verilerin şeklini verir

((95500, 13), (40929, 13))

In [18]:
from sklearn.preprocessing import MinMaxScaler          #veriyi daha rahat işlemesi için bir aralığa sıkıştırır veri 0 ile 1 arasında olur
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val= scaler.transform(X_val)

In [19]:
xgb_model = xgb.XGBClassifier()                                                                     #veri XGBClassifier kullanılarak eğitilir ve sonuç print edilir
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'XGBoost Cross-Validation: {xgb_scores.mean():.4f}')

rf_model = RandomForestClassifier()                                                                 #veri RandomForestClassifier kullanılarak eğitilir ve sonuç print edilir
rf_scores = cross_val_score(rf_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Random Forest Cross-Validation: {rf_scores.mean():.4f}')

knn_model = KNeighborsClassifier()                                                                  #veri KNeighborsClassifier kullanılarak eğitilir ve sonuç print edilir
knn_scores = cross_val_score(knn_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'KNN Cross-Validation: {knn_scores.mean():.4f}')

logreg_model = LogisticRegression(class_weight = 'balanced')                                        #veri LogisticRegression kullanılarak eğitilir ve sonuç print edilir
logreg_scores = cross_val_score(logreg_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Logistic Regression Cross-Validation: {logreg_scores.mean():.4f}')

dt_model = DecisionTreeClassifier()                                                                 #veri DecisionTreeClassifier kullanılarak eğitilir ve sonuç print edilir
dt_scores = cross_val_score(dt_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'Decision Tree Cross-Validation: {dt_scores.mean():.4f}')

svm_model = SVC(probability=True)                                                                   #veri SVC kullanılarak eğitilir ve sonuç print edilir
svm_scores = cross_val_score(svm_model, X_train, y_train, cv=5, scoring='roc_auc_ovo')
print(f'SVM Cross-Validation: {svm_scores.mean():.4f}')

XGBoost Cross-Validation: 0.9526
Random Forest Cross-Validation: 0.9398
KNN Cross-Validation: 0.8897
Logistic Regression Cross-Validation: 0.9333
Decision Tree Cross-Validation: 0.8817
SVM Cross-Validation: 0.9191


In [20]:
xgb_model = xgb.XGBClassifier()                                                                     #veri XGBClassifier kullanılarak eğitilir ve sonuç print edilir
xgb_scores = cross_val_score(xgb_model, X_train, y_train, cv=500, scoring='roc_auc_ovo')
print(f'XGBoost Cross-Validation: {xgb_scores.mean():.4f}')

XGBoost Cross-Validation: 0.9583


In [21]:
Y_predict=xgb_model.predict(data_test)

NotFittedError: need to call fit or load_model beforehand